In [252]:
%run fuzzy_pakar_csv.ipynb

RULE 1: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal kecil (0.0000) AND nilai kecil (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 2: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal kecil (0.0000) AND nilai kecil (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 3: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal kecil (0.0000) AND nilai kecil (1.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 4: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal sedang (0.0000) AND nilai sedang (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 5: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal sedang (0.0000) AND nilai sedang (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 6: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal sedang (0.0000) AND nilai sedang (1.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 7: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal besar (1.0000) AND nilai bes

In [253]:
class Particle:
    def __init__(self, bounds):
        self.bounds = bounds
        self.position = self._generate_position()
        self.velocity = self._generate_velocity()
        self.best_position = np.copy(self.position)
        self.best_score = -float('inf')

    def _generate_position(self):
        position = []
        for min_bound, max_bound, length in self.bounds:
            segment = np.random.uniform(min_bound, max_bound, size=length)
            position.extend(segment)
        return np.array(position)

    def _generate_velocity(self):   
        velocity = []
        for _, _, length in self.bounds:
            segment = np.random.uniform(-1, 1, size=length)
            velocity.extend(segment)
        return np.array(velocity)

    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.rand(len(self.position))
        r2 = np.random.rand(len(self.position))
        cognitive = c1 * r1 * (self.best_position - self.position)
        social = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive + social

    def update_position(self):
        self.position += self.velocity
        start_idx = 0
        for min_bound, max_bound, length in self.bounds:
            end_idx = start_idx + length
            self.position[start_idx:end_idx] = np.clip(
                self.position[start_idx:end_idx], min_bound, max_bound
            )
            start_idx = end_idx

def objective_function(particle_position, dataset):
    prediksi_benar = 0
    for i in range(len(dataset)):
        waktu_belajar = dataset.loc[i, 'rerata_durasi_akses_materi']
        lama_jawab_soal = dataset.loc[i, 'durasi_berlatih']
        nilai = dataset.loc[i, 'nilai']
        aktual_kelulusan = dataset.loc[i, 'kelulusan']
        x, y, z = func_fuzzifikasi(waktu_belajar, lama_jawab_soal, nilai, particle_position)
        print(f"waktu belajar   : {x}")
        print(f"lama jawab soal : {y}")
        print(f"nilai           : {z}")
        hasil_inferensi, rule_text = inferensi_mamdani(x, y, z, rules)
        hasil_agregasi = agregasi(hasil_inferensi)
        hasil_defuzzifikasi = defuzzifikasi(hasil_agregasi)
        prediksi_kelulusan = kelulusan(hasil_defuzzifikasi)
        if int(prediksi_kelulusan) == int(aktual_kelulusan):
            prediksi_benar += 1
        else:
            continue
    akurasi = (prediksi_benar / len(dataset)) * 100
    return akurasi

def particle_swarm_optimization(bounds, dataset, num_particles, max_iter, w=0.5, c1=2, c2=2):
    swarm = []
    for _ in range(num_particles):
        swarm.append(Particle(bounds))
    global_best_position = None
    global_best_score = -float('inf')

    for iteration in range(max_iter):
        for particle in swarm:
            # Evaluate fitness
            print(particle.position)
            # print(len(particle.velocity))
            particle_score = objective_function(particle.position, dataset)
            # print(particle_score)
            if particle_score > particle.best_score:
                particle.best_score = particle_score
                particle.best_position = np.copy(particle.position)

            if particle_score > global_best_score:
                global_best_score = particle_score
                global_best_position = np.copy(particle.position)

        # Update velocity and position
        for particle in swarm:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position(bounds)

        print(f"Iteration {iteration+1}/{max_iter}, Best Score: {global_best_score}")

    return global_best_position, global_best_score


bounds = [
    (0, 3600, 11),  # Elemen 0-10 dengan batas [0, 1]
    (0, 9000, 11),  # Elemen 11-20 dengan batas [1, 5]
    (0, 100, 10)  # Elemen 21-31 dengan batas [-3, 3]
]

# Run PSO
num_particles = 10
max_iter = 20
best_params, best_score = particle_swarm_optimization(bounds, dataset, num_particles, max_iter)

print("\nOptimal Parameters:")
print(f"C: {best_params[0]:.4f}, Gamma: {best_params[1]:.4f}")
print(f"Best Cross-Validation Accuracy: {best_score:.4f}")

[5.28213277e+02 2.54561648e+03 1.82927404e+03 2.04944631e+03
 2.59676042e+03 2.66850624e+03 1.25727175e+03 2.66186113e+03
 2.55110923e+03 2.03779204e+03 3.52342164e+03 2.33892251e+03
 4.30132379e+03 5.06324654e+03 7.98754865e+03 3.63066004e+03
 3.84525358e+03 5.24365051e+03 1.98001139e+03 3.33378942e+03
 1.70241256e+03 6.15510314e+03 5.92820289e+01 8.09825966e+01
 3.32236333e-01 3.74905658e+01 8.13533570e+01 4.96486614e+01
 4.48431940e+00 1.66152799e+01 8.43239361e+00 6.51673420e+01]
waktu belajar   : {'kecil': 0.030626858640352434, 'sedang': 0, 'besar': 0}
lama jawab soal : {'kecil': 0, 'sedang': 0, 'besar': 0}
nilai           : {'kecil': 0, 'sedang': 0, 'besar': 0}


ZeroDivisionError: float division by zero